# Portfolio Project: Safety Option

**Names:** Deniz Makul, Richard Sha, Mitchel Shen

**Due:** 11/25/2022 at 11:59pm ET on Dropbox

**What to submit:** your .ipynb (python code in jupyter notebooks) file as well as a pdf with all the output from your code.  




In [184]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
from threading import Thread
import time

In [185]:
#Getting tickers from csv file
csv_tickers = pd.read_csv('Tickers_Example.csv', header = None)

In [186]:
# Returns a list of of months that have less than 20 trading days

volume_start_date = '2022-01-01'
volume_end_date = '2022-10-31'

def remove_month(ticker):
    hist = ticker.history(start = volume_start_date, end = volume_end_date)
    
    months_to_remove = []
    for i in range(1,11,1):
        if i <= 9: #appends a 0 if single digit
            i = str(0) + str(i)
        df = hist.filter(like='2022-' + str(i), axis=0)
        if len(df.index) < 20:
            months_to_remove.append('2022-' + str(i))
    return months_to_remove

In [187]:
volume_index_drop = []

def filter_stocks(ticker, i):
    stock_data = yf.Ticker(ticker)
    stock_hist = stock_data.history(start = volume_start_date, end = volume_end_date, interval = '1mo', Threads = True)
    
    if stock_hist.empty:
        volume_index_drop.append(i)
        return
    
    stock_volume = stock_hist['Volume']
    stock_volume = stock_volume.dropna()
    stock_volume = stock_volume.drop(remove_month(stock_data))
    
    mean = (stock_volume.mean())

    if mean<200000:
        volume_index_drop.append(i)         
    elif(stock_data.info['market'] != 'us_market'):
        volume_index_drop.append(i)
    
    #volume_index_drop

In [188]:
#Drops the tickers that are delisted or have a average monthly volume from 
# January 1st to October 31st of less that 200,000

# Sending in the first ticker to determine which month to remove

thread_list = []

for i in range(len(csv_tickers)):
    ticker = csv_tickers[0][i]
    filter_thread = Thread(target = filter_stocks, args = (ticker,i,))
    thread_list.append(filter_thread)
    filter_thread.start()
    
for i in thread_list:
    i.join()

print(volume_index_drop)

- AGN: No data found, symbol may be delisted
- CELG: No data found, symbol may be delisted
- RTN: No data found, symbol may be delisted
- PCLN: No data found for this date range, symbol may be delisted
- TWX: No data found for this date range, symbol may be delisted
[4, 16, 47, 40, 56, 54, 48]


In [189]:
#Drops invalid tickers
csv_tickers.drop(index = volume_index_drop, inplace = True)
csv_tickers.reset_index(drop = True, inplace = True)
csv_tickers.head()

,0
0,AAPL
1,ABBV
2,ABT
3,ACN
4,AIG


In [190]:
tickers_list = []
price_list = []
shares_list = []
values_list = []
weight_list = []

In [191]:
# Richard
total_amount_to_spend = 500000

Portfolio_Final = pd.DataFrame({
    'Ticker': tickers_list,
    'Price': price_list,
    'Shares': shares_list,
    'Value': values_list,
    'Weight': weight_list
})

In [192]:
# Deniz

In [193]:
# Mitchel
close_start = '2022-01-01'
close_end = '2022-10-31'
close_all = pd.DataFrame()
for i in range(0,len(csv_tickers)):
    stock_data = yf.Ticker(csv_tickers[0][i])
    stock_hist = stock_data.history(start = close_start, end = close_end, interval = '1d')
    stock_close = stock_hist['Close']
    stock_close = stock_close.dropna()
    close_all[csv_tickers[0][i]] = stock_close
    
close_all.head()

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,SLB,SO,SPG,T,TGT,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,182.009995,134.029205,138.551285,406.156982,57.389999,170.404495,167.788895,207.860001,45.955307,244.139999,...,31.621540,67.475571,160.300003,18.700493,230.949112,190.600006,502.279999,246.562317,211.959869,57.270000
2022-01-04,179.699997,133.771881,135.292801,403.254517,59.000000,167.522003,173.175369,213.630005,47.756504,241.729996,...,33.156757,67.475571,162.979996,18.854919,233.159531,190.800003,490.899994,250.801544,215.722519,58.770000
2022-01-05,174.919998,134.474579,134.684937,396.152924,58.189999,164.356995,171.329987,213.070007,46.950443,239.270004,...,33.156757,67.554756,161.029999,19.274082,227.872452,186.820007,489.690002,251.557831,213.012222,58.459999
2022-01-06,172.000000,133.841156,134.665009,377.022522,59.189999,163.253998,172.899994,211.339996,47.895821,237.300003,...,33.944305,67.475571,163.160004,19.200544,230.351715,186.759995,469.649994,252.254440,215.077225,60.049999
2022-01-07,172.169998,133.494751,135.083527,369.791290,60.740002,162.554001,174.380005,215.500000,48.940712,232.600006,...,34.921265,67.940781,161.270004,19.856495,229.784164,179.440002,458.600006,253.538147,216.586258,61.000000


In [194]:
close_all_pct = close_all.pct_change() * 100
close_all_pct.head()

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,SLB,SO,SPG,T,TGT,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-04,-1.269160,-0.191991,-2.351825,-0.714617,2.805368,-1.691559,3.210269,2.775909,3.919454,-0.987140,...,4.854973,0.000000,1.671861,0.825789,0.957102,0.104930,-2.265670,1.719333,1.775171,2.619172
2022-01-05,-2.659988,0.525296,-0.449295,-1.761070,-1.372884,-1.889309,-1.065615,-0.262134,-1.687856,-1.017661,...,0.000000,0.117354,-1.196464,2.223095,-2.267580,-2.085952,-0.246484,0.301548,-1.256381,-0.527482
2022-01-06,-1.669334,-0.471035,-0.014796,-4.829045,1.718508,-0.671098,0.916365,-0.811945,2.013564,-0.823338,...,2.375226,-0.117217,1.322738,-0.381537,1.088005,-0.032123,-4.092387,0.276918,0.969429,2.719809
2022-01-07,0.098836,-0.258818,0.310785,-1.917984,2.618691,-0.428778,0.855993,1.968394,2.181592,-1.980614,...,2.878124,0.689449,-1.158372,3.416312,-0.246384,-3.919465,-2.352813,0.508894,0.701624,1.582016


In [195]:
#Expected Return
expected_return = close_all_pct.mean()
expected_return.head()

AAPL   -0.051033
ABBV    0.057890
ABT    -0.145114
ACN    -0.146202
AIG     0.019107
dtype: float64

In [196]:
expected_return_from_0 = abs(0 - expected_return)
expected_return_from_0

expected_return_sorted = sorted(expected_return_from_0.items(), key=lambda x:x[1])

In [197]:
std_lst = close_all_pct.std()
std_lst = sorted(std_lst.items(), key=lambda x:x[1])

In [199]:
correlation_pct_stocks = pd.DataFrame()
index = []

for i in range(len(close_all_pct.columns)):
    
    correlation_temp = {}
    
    for j in range(len(close_all_pct.columns)):
        correlation_df = pd.DataFrame({
            csv_tickers[0][i]: close_all_pct[csv_tickers[0][i]]
        })
        
        correlation_df[csv_tickers[0][j]] = close_all_pct[csv_tickers[0][j]]
        correlation = correlation_df.corr()[csv_tickers[0][i]][csv_tickers[0][j]]

        correlation_temp[csv_tickers[0][j]] = correlation
        
    correlation_pct_stocks[csv_tickers[0][i]] = list(correlation_temp.values())
    index.append(csv_tickers[0][i])


correlation_pct_stocks.set_index([pd.Index(index)])
correlation_pct_stocks.head()

,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,...,SLB,SO,SPG,T,TGT,TXN,UNH,UNP,UPS,USB
0,1.000000,0.196115,0.607975,0.788189,0.564777,0.661361,0.640823,0.576638,0.566142,0.298499,...,0.208881,0.370039,0.596782,0.378568,0.506691,0.720272,0.518906,0.474990,0.536375,0.471165
1,0.196115,1.000000,0.448482,0.310854,0.300005,0.195382,0.254210,0.103088,0.287487,0.222762,...,0.168036,0.369601,0.246472,0.277107,0.259999,0.235616,0.523391,0.242589,0.201065,0.368628
2,0.607975,0.448482,1.000000,0.685228,0.530735,0.486366,0.580548,0.428488,0.563878,0.328822,...,0.117035,0.438613,0.513648,0.367974,0.422302,0.523594,0.591601,0.490681,0.468793,0.542299
3,0.788189,0.310854,0.685228,1.000000,0.600418,0.677280,0.649058,0.548413,0.589600,0.384701,...,0.203834,0.411081,0.640848,0.365729,0.547363,0.735174,0.581976,0.556123,0.574278,0.527163
4,0.564777,0.300005,0.530735,0.600418,1.000000,0.456730,0.737495,0.534551,0.757113,0.241459,...,0.395121,0.290292,0.601893,0.372322,0.340778,0.489535,0.400360,0.512507,0.509901,0.732907


In [200]:
correlation_pct_mean = pd.DataFrame()
correlation_pct_mean = correlation_pct_stocks.sum()
correlation_pct_mean = correlation_pct_mean - 1
correlation_pct_mean = correlation_pct_mean/(len(correlation_pct_stocks) - 1)

correlation_pct_mean.head()

AAPL    0.497130
ABBV    0.284643
ABT     0.467452
ACN     0.523949
AIG     0.473681
dtype: float64

In [201]:
correlation_from_0 = abs(0 - correlation_pct_mean)
correlation_from_0 = sorted(correlation_from_0.items(), key=lambda x:x[1])

In [202]:
points_system = pd.DataFrame()
expected_lst = []
std_deviation_lst = []
correlation_lst = []

for i in expected_return_sorted:
    expected_lst.append(i[0])
    
for i in std_lst:
    std_deviation_lst.append(i[0])

for i in correlation_from_0:
    correlation_lst.append(i[0])

points_system['Expected Returns'] = expected_lst
points_system['Standard Deviation'] = std_deviation_lst
points_system['Correlation'] = correlation_lst
points_system.index += 1 

points_system = points_system.set_index([points_system.index[::-1]])

points_system.head()

,Expected Returns,Standard Deviation,Correlation
55,SO,MON,MON
54,PM,BMY,LMT
53,MO,MRK,OXY
52,MON,PEP,SLB
51,T,KO,COP


In [203]:
score = pd.DataFrame()
    
score['Tickers'] = expected_lst

score['Points'] = None

for i in range(len(score)):
    score['Points'][i] = (int(points_system[points_system['Expected Returns'] == expected_lst[i]].index.values)) + (int(points_system[points_system['Standard Deviation'] == expected_lst[i]].index.values)) + (int(points_system[points_system['Correlation'] == expected_lst[i]].index.values)) 

score = score.sort_values("Points")
score = score[::-1]
score = score.reset_index(drop=True)
score.index += 1 

score.head()

,Tickers,Points
1,MON,162
2,SO,147
3,MO,144
4,PM,142
5,ABBV,134


In [204]:
number_of_stocks = 25

score_dropped = score.copy()
score_dropped.drop(labels = range(number_of_stocks + 1, len(score) + 1), inplace = True)
score_dropped.head()

,Tickers,Points
1,MON,162
2,SO,147
3,MO,144
4,PM,142
5,ABBV,134


In [205]:
min_weighting = 100/(2*number_of_stocks)
base_weighting_total = min_weighting * number_of_stocks

score_dropped['Base Weighting'] = min_weighting
weighting_sum = score_dropped['Points'].sum()

score_dropped['Points-based Weighting'] = (score_dropped['Points']/weighting_sum) * (100 - base_weighting_total)
score_dropped['Total Weighting'] = score_dropped['Base Weighting'] + score_dropped['Points-based Weighting']
score_dropped.head()

,Tickers,Points,Base Weighting,Points-based Weighting,Total Weighting
1,MON,162,2.0,2.779684,4.779684
2,SO,147,2.0,2.522306,4.522306
3,MO,144,2.0,2.47083,4.47083
4,PM,142,2.0,2.436513,4.436513
5,ABBV,134,2.0,2.299245,4.299245
